In [ ]:
class CO2f_Takahashi_comp:
    '''
    class CO2f_Takahashi_comp(resultpath,savepath,meshpath,txtfile,first_year,last_year,
                 mapproj='pc',savefig=False,layerwise=False,runname='fesom')
    '''
    def __init__(self,resultpath,savepath,mesh,txtfile,first_year,last_year,
                 mapproj='rob',
                 cmap='RdBu_r',
                 savefig=False,
                 cmap_extension='both',
                 verbose=False,
                 plotting=True,
                 output=False,
                 Taylor=True,runname='fesom'):

        self.runname = runname
        self.resultpath = resultpath
        self.savepath = savepath
        self.mesh = mesh
        self.txtfile = txtfile
        self.fyear = first_year
        self.lyear = last_year
        self.mapproj = mapproj
        self.savefig = savefig
        self.cmap = cmap
        self.verbose = verbose
        self.plotting = plotting
        self.output = output
        self.Taylor = Taylor
        self.cmap_extension = cmap_extension
        
        import matplotlib.pyplot as plt
        import numpy as np
        from netCDF4 import Dataset
        from scipy.interpolate import griddata
        import skill_metrics as sm
        import cartopy.crs as ccrs
        import pyfesom2 as pf
        from plot_Taylor_normalized import plt_Taylor_norm
        
        if self.mapproj == 'rob':
            box=[-180, 180, -90, 90]
        elif self.mapproj == 'pc':
            box=[-180, 180, -90, 90]
        elif self.mapproj == 'sp':
            box=[-180, 180, -90, -30]
        elif self.mapproj == 'np':
            box=[-180, 180, 60, 90]
            
        self.mapproj = pf.get_proj(self.mapproj)
        years = np.arange(self.fyear, self.lyear+1,1)

        # load FESOM data -------------------------------------------------------------------------------------
        CO2ffesom = pf.get_data(resultpath, "CO2f", years, mesh, 
                               how="mean", compute=True, runid=self.runname, silent=True)    

        CO2ffesom = -365.*CO2ffesom/1000. # converting to molC/m2/year
    
    
        labelfesom = 'FESOM ({0}-{1})'.format(self.fyear,self.lyear)
        unitfesom = 'air-sea CO$_2$ flux [mol C m$^{-2}$ yr$^{-1}$]' 

        # Reading CO2 flux data from Takahashi =========================================
        labeltakashi = 'Takahashi et al.'
        path = self.txtfile
        header = 109                                          # Number of lines in file to skip (with text)
        allvalues = []
        with open(path, 'r') as f:
            for _ in range(header):                             # Skipping text lines
                f.readline()
            for line in f:                                      # Saving all data in matrix "allvalues"
                allvalues.append(line.split())

        allvalues     = np.array(allvalues)                   # Converting list to array
        allvalues     = allvalues.astype(float)            # Converting string to float
        lat           = allvalues[:,0]                        # Saving all latitude values in array "lat"
        lat_matrix    = np.unique(lat)                           # 
        lon           = allvalues[:,1]                        # Saving all longitude values in array "lon"
        lon[lon>180.] = lon[lon>180.]-360.                    # Converting lon to values from -180 to 180
        lon_matrix    = np.unique(lon)                           #
        month         = allvalues[:,2]                        # Saving number of month in array "month"      
        flux          = allvalues[:,17]                       # CO2 flux in units [moles C/m2/month] saved in array "flux"

        matrix = np.zeros(shape=(72,40,12))                   # Initializing matrix [lon,lat,month] for gridded sorting of flux values
        for i in range(0,len(flux)):                          # Stepping through all values and adding them to proper position in the matrix
            n = np.where(lat_matrix == lat[i])[0] 
            m = np.where(lon_matrix == lon[i])[0]
            matrix[m,n,int(month[i]-1)] = flux[i]
        matrix[matrix==0] = np.nan                                 # Converting missing values to NaN
        CO2matrix = np.sum(matrix,axis=2)                     # Yearly CO2 flux [moles C/m2/yr]
        CO2matrix = CO2matrix.transpose()                     # Transposing 

        # ==============================================================================
        # Interpolation to fesom's grid

        lonbefore, latbefore = np.meshgrid(lon_matrix, lat_matrix)               # Matrix lon and lat prepared for interpolation

        CO2ftakashi = griddata((lonbefore.ravel(), latbefore.ravel()), CO2matrix.ravel(), (mesh.x2, mesh.y2), method='nearest')
        CO2ftakashi = np.ma.filled(CO2ftakashi, np.nan)

        # apply sea mask to Takashi as in FESOM ----------------------------------------------------------------------------------
        # assumption: there is no ocean where value in FESOM == 0
        CO2ftakashi_ma = np.copy(CO2ftakashi)
        CO2ftakashi_ma[np.isnan(CO2ffesom)] = np.nan
        
        
        # Plotting -------------------------------------------------------------------------------------
        if(self.verbose):
            print('\nPlotting CO2 flux \nFESOM min = {0}, max = {1}\nTakashi min = {2}, max = {3}'.format(
                    np.nanmin(CO2ffesom),np.nanmax(CO2ffesom),
                    np.nanmin(CO2ftakashi_ma),np.nanmax(CO2ftakashi_ma)))
        
        if plotting:
                fig = plt.figure(figsize=(15,12), constrained_layout=False)
                axes = fig.subplot_mosaic(
                                """
                                AB
                                CC
                                """,
                                gridspec_kw={'hspace': 0.1, 'wspace': 0.1, 'bottom': 0.03}, 
                                subplot_kw=dict(projection=self.mapproj))
                    
                m1 = axes['A']
                levels = np.arange(-9,9.2,.2)
                f1 = pf.subplot(mesh, fig, m1, [CO2ffesom],
                            levels = levels,
                            units=unitfesom, 
                            mapproj=self.mapproj, # robinson projection takes more time!
                            cmap = self.cmap,
                            cmap_extension=self.cmap_extension,
                            titles=labelfesom,
                            box=box,
                           )
                    
                m2 = axes['B']
                f2 = pf.subplot(mesh, fig, m2, [CO2ftakashi_ma], 
                            levels = levels,
                            units=unitfesom, 
                            mapproj=self.mapproj, # robinson projection takes more time!
                            cmap = self.cmap,
                            cmap_extension=self.cmap_extension,
                            titles=labeltakashi,
                            box=box,
                           )
                    
                cbar1_ax = fig.add_axes([0.13, 0.53, 0.76, 0.02])
                cbar1 = fig.colorbar(f1,
                                cax = cbar1_ax, 
                                orientation = 'horizontal',
                                fraction=0.046, pad=0.04)
                cbar1.set_label(unitfesom, fontsize=18)
                cbar1.ax.tick_params(labelsize=18)
        
                m3 = axes['C']

                levels_diff = np.arange(-5,5.2,.2)
                f3 = pf.subplot(mesh, fig, m3, [CO2ffesom-CO2ftakashi_ma], 
                            rowscol= (1,1),
                            levels = levels_diff,
                            units=unitfesom, 
                            mapproj=self.mapproj, # robinson projection takes more time!
                            cmap = 'RdBu_r',
                            cmap_extension='both',
                            titles='FESOM - Takahashi',
                            box=box,
                           )
                
                fig.subplots_adjust(bottom=0.02)
                cbar2_ax = fig.add_axes([0.13, 0.001, 0.76, 0.02])
                cbar2 = fig.colorbar(f3,
                                cax = cbar2_ax, 
                                orientation = 'horizontal',
                                fraction=0.046, pad=0.04) 
                cbar2.set_label(unitfesom, fontsize=18)
                cbar2.ax.tick_params(labelsize=18)

                m1.text(-0.12, 1.05, 'A', transform=m1.transAxes,
                            size=30, weight='bold')
                m2.text(-0.12, 1.05, 'B', transform=m2.transAxes,
                            size=30, weight='bold')
                m3.text(-0.12, 1.05, 'C', transform=m3.transAxes,
                            size=30, weight='bold')

                # fig export  -------------------------------------------------------------------------------------
                if(self.savefig==True):
                    plt.savefig(self.savepath+self.runname+'_'+'CO2f_Taka'+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                            dpi = 300, bbox_inches='tight')
                    plt.savefig(self.savepath+self.runname+'_'+'CO2f_Taka'+'_'+str(years[0])+'to'+str(years[-1])+'.pdf', 
                            bbox_inches='tight')
                plt.show(block=False)

        # statistics  -------------------------------------------------------------------------------------            
        # preparation of datasets

        if Taylor:
            #get statistics only from valid Takashi points
            ind_stat = np.where(np.isfinite(CO2ftakashi_ma))

            if np.isnan(np.min(CO2ftakashi_ma[ind_stat])): print('WARNING: The interpolated Takashi field contains NaNs at depth')
            if np.isnan(np.min(CO2ffesom[ind_stat])): print('WARNING: The FESOM field contains NaNs at depth')

            title = 'Taylor Diagram for CO$_2$ flux'
            plt_Taylor_norm(CO2ftakashi_ma[ind_stat],CO2ffesom[ind_stat],mask=True,title=title)

            # fig export  -------------------------------------------------------------------------------------
            if(self.savefig==True):                
                plt.savefig(self.savepath+self.runname+'_'+'CO2flux_Takashi_Taylor'+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                        dpi = 300, bbox_inches='tight')
                plt.savefig(self.savepath+self.runname+'_'+'CO2flux_Takashi_Taylor'+'_'+str(years[0])+'to'+str(years[-1])+'.pdf', 
                        bbox_inches='tight')
                plt.show(block=False)
            
        if output:
                    self.co2f_fesom = CO2ffesom
                    self.co2f_taka  = CO2ftakashi_ma